# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [149]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key
geoapify_key

'fb8f7ecec6e2453988716af18c7bba24'

In [110]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv('cities.csv')

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nikolskoye,59.7035,30.7861,-4.06,93,89,0.49,RU,1674461278
1,1,mataura,-46.1927,168.8643,11.33,71,7,1.88,NZ,1674461279
2,2,vaini,-21.2000,-175.2000,26.09,94,20,0.00,TO,1674461279
3,3,albany,42.6001,-73.9662,0.36,93,100,1.68,US,1674461280
4,4,new norfolk,-42.7826,147.0587,22.72,85,100,1.13,AU,1674461280


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [150]:
%%capture --no-display

# Configure the map plot
coordinates = {

"Lat": (city_data_df["Lat"]),
"Lng": (city_data_df["Lng"]),
"City": (city_data_df["City"]),
"Humidity": (city_data_df["Humidity"])
}

coordinates_df = pd.DataFrame(coordinates)
coordinates_df

# Display the map
humidity_map = coordinates_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [146]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df.loc[(city_data_df["Max Temp"] >= 10) & 
                            (city_data_df["Max Temp"] < 30).dropna()]
ideal_weather


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,mataura,-46.1927,168.8643,11.33,71,7,1.88,NZ,1674461279
2,2,vaini,-21.2000,-175.2000,26.09,94,20,0.00,TO,1674461279
4,4,new norfolk,-42.7826,147.0587,22.72,85,100,1.13,AU,1674461280
6,6,butaritari,3.0707,172.7902,27.28,77,99,5.58,KI,1674461280
8,8,kieta,-6.2167,155.6333,26.83,80,100,1.34,PG,1674461280
...,...,...,...,...,...,...,...,...,...,...
554,554,tombouctou,20.0000,-3.0000,15.71,15,0,4.39,ML,1674461414
558,558,kabo,7.6994,18.6290,29.54,15,100,1.86,CF,1674461415
559,559,ngunguru,-35.6167,174.5000,18.95,77,65,1.42,NZ,1674461415
563,563,the valley,18.2170,-63.0578,25.05,72,14,10.34,AI,1674461416


### Step 3: Create a new DataFrame called `hotel_df`.

In [147]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
hotel_df["Hotel Name"] = " "
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
1,mataura,NZ,-46.1927,168.8643,71,
2,vaini,TO,-21.2000,-175.2000,94,
4,new norfolk,AU,-42.7826,147.0587,85,
6,butaritari,KI,3.0707,172.7902,77,
8,kieta,PG,-6.2167,155.6333,80,
...,...,...,...,...,...,...
554,tombouctou,ML,20.0000,-3.0000,15,
558,kabo,CF,7.6994,18.6290,15,
559,ngunguru,NZ,-35.6167,174.5000,77,
563,the valley,AI,18.2170,-63.0578,72,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [148]:
# Set parameters to search for a hotel
radius = 10000
categories = ["accomodation.hotel"]
params = {"categories": categories,
"apiKey" : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mataura - nearest hotel: No hotel found
vaini - nearest hotel: No hotel found
new norfolk - nearest hotel: No hotel found
butaritari - nearest hotel: No hotel found
kieta - nearest hotel: No hotel found
puerto carreno - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
rikitea - nearest hotel: No hotel found
camacha - nearest hotel: No hotel found
cape town - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
nuevo progreso - nearest hotel: No hotel found
olinda - nearest hotel: No hotel found
hermanus - nearest hotel: No hotel found
laguna de perlas - nearest hotel: No hotel found
faya - nearest hotel: No hotel found
innisfail - nearest hotel: No hotel found
kapaa - nearest hotel: No hotel found
punta arenas - nearest hotel: No hotel found
hobart - nearest hotel: No hotel found
riyadh - nearest hotel: No hotel found
arlit - nearest hotel: No hotel found
chegutu - nearest hotel: No hotel found
lebu - nearest hotel: No 

,City,Country,Lat,Lng,Humidity,Hotel Name
1,mataura,NZ,-46.1927,168.8643,71,No hotel found
2,vaini,TO,-21.2000,-175.2000,94,No hotel found
4,new norfolk,AU,-42.7826,147.0587,85,No hotel found
6,butaritari,KI,3.0707,172.7902,77,No hotel found
8,kieta,PG,-6.2167,155.6333,80,No hotel found
...,...,...,...,...,...,...
554,tombouctou,ML,20.0000,-3.0000,15,No hotel found
558,kabo,CF,7.6994,18.6290,15,No hotel found
559,ngunguru,NZ,-35.6167,174.5000,77,No hotel found
563,the valley,AI,18.2170,-63.0578,72,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)